In [ ]:
#필요 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install pandas
#KoBERT 깃허브에서 불러오기
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 33.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619631 sha256=28179ae816503cf1eb785535a086b6c9201f5c5c91143eda40fa59590308c098
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
#KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 설정
device = torch.device("cuda:0")

In [ ]:
#bertmodel의 vocabulary
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
train_data = pd.read_csv('/content/drive/Shareddrives/인지프/data/nlp/감성대화_전처리/train_data_final')
train_data2 = pd.read_csv('/content/drive/Shareddrives/인지프/data/nlp/감성대화_전처리/traindata_그런데추가데이터인(4차제거)')
test_data = pd.read_csv('/content/drive/Shareddrives/인지프/data/nlp/감성대화_전처리/test_data_final')

In [ ]:
train_data.sample(n=10)

,Unnamed: 0,감정_대분류,사람문장1
8136,8136,불안,혹시 내가 치매를 앓고 있다는 것을 가족들이 알면 버릴까?
17527,17527,상처,내 재산을 자식들에게 물려주지 않겠다고 하니 날 찾아와서는 화를 내다니 배신감이 느껴져.
25622,25622,당황,항상 날 괴롭히던 애가 집이 잘살아서 내가 너무 비참해.
30314,30314,슬픔,남들 앞에서 약한 모습을 보이기가 싫어서 억지로 괜찮은 척 웃는 나 자신이 미워.
23850,23850,불안,학교폭력 가해자로 찍혀서 내 미래를 망치지 않을까?
32558,32558,상처,이렇게 차일 줄은 몰랐는데 너무 마음이 아파.
9220,9220,당황,친구가 헤어진 사실을 말하지 말라고 해서 안 했는데 며칠 뒤에 왜 말했느냐며 화를 냈어.
40367,40367,분노,열심히 살아 온 내가 하필 암이라는 무서운 병에 걸리다니! 너무 화가 나서 견딜 수...
586,586,기쁨,우리 가족은 너무 믿음직스러운 것 같아.
39367,39367,슬픔,직장 다니는 일이 쉬운 일이 아니야. 갈수록 새로운 기술에 따른 업무를 배우는 것이...


In [ ]:
train_data.isnull().sum()

Unnamed: 0    0
감정_대분류        0
사람문장1         0
dtype: int64

In [ ]:
train_data2.isnull().sum()

Unnamed: 0    0
발화문           0
상황            0
dtype: int64

In [ ]:
train_data.loc[(train_data['감정_대분류'] == "불안"), '감정_대분류'] = 0  #불안 => 0
train_data.loc[(train_data['감정_대분류'] == "불안 "), '감정_대분류'] = 0  #불안 => 0
train_data.loc[(train_data['감정_대분류'] == "당황"), '감정_대분류'] = 1  #당황 => 1
train_data.loc[(train_data['감정_대분류'] == "분노"), '감정_대분류'] = 2  #분노 => 2
train_data.loc[(train_data['감정_대분류'] == "슬픔"), '감정_대분류'] = 3  #슬픔 => 3
train_data.loc[(train_data['감정_대분류'] == "상처"), '감정_대분류'] = 3  #상처 => 슬픔 -> 3
train_data.loc[(train_data['감정_대분류'] == "기쁨"), '감정_대분류'] = 4  #기쁨 => 4
train_data.loc[(train_data['감정_대분류'] == "기쁨 "), '감정_대분류'] = 4  #기쁨 => 4

In [ ]:
test_data.loc[(test_data['감정_대분류'] == "불안"), '감정_대분류'] = 0  #불안 => 0

test_data.loc[(test_data['감정_대분류'] == "당황"), '감정_대분류'] = 1  #당황 => 1
test_data.loc[(test_data['감정_대분류'] == "분노"), '감정_대분류'] = 2  #분노 => 2
test_data.loc[(test_data['감정_대분류'] == "슬픔"), '감정_대분류'] = 3  #슬픔 => 3
test_data.loc[(test_data['감정_대분류'] == "상처"), '감정_대분류'] = 3  #상처 => 슬픔 -> 3
test_data.loc[(test_data['감정_대분류'] == "기쁨"), '감정_대분류'] = 4  #기쁨 => 4


In [ ]:
import numpy as np



In [ ]:
train_data_list = []
for q,  label in zip(train_data['사람문장1'], train_data['감정_대분류'])  :
    data = []
    data.append(q)
    
    data.append(str(label))

    train_data_list.append(data)

In [ ]:
for q,  label in zip(train_data2['발화문'], train_data2['상황'])  :
    data = []
    data.append(q)
    
    data.append(str(label))

    train_data_list.append(data)

In [ ]:
test_data_list = []
for q, label in zip(test_data['사람문장1'], test_data['감정_대분류'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

In [ ]:
print(len(train_data_list))

68992


In [ ]:
import random

random.shuffle(train_data_list)

In [ ]:
test_data_list = train_data_list[:6001]
train_data_list = train_data_list[6001:]

In [ ]:
dataset_test = pd.DataFrame(test_data_list)
dataset_test

,0,1
0,내가 빨리 달려가서 해피를 안고왔기 때문에 아무 일도 없었어.,1
1,요즘 룸메이트랑 너무 자주 싸우게 돼.,2
2,연로하신 시어머니께서 힘든 항암치료를 버티실 수 있을까 모르겠어. 너무 불안해.,0
3,지금 아들이 교통사고를 당했는데 일일구가 빨리 오지 않아! 답답해.,2
4,드디어 구매한 노트북이 도착했어.,4
...,...,...
5996,어려운 수학 숙제를 이제야 겨우 끝냈어.,4
5997,새로운 여자친구를 사귀었는데 진짜 마음에 들어.,4
5998,매우 떨어지고 있어.,3
5999,남편 벌이가 시원치 않아. 나도 남들처럼 돈 잘 버는 남편 만나 여생 피고 싶어.,3


In [ ]:
dataset_train = pd.DataFrame(train_data_list)
dataset_train

,0,1
0,나 너무 우울해.,3
1,부모님이 내가 하고 싶은 게 고작 그런 일이냐면서 그만두고 가게 일이나 도우래.,3
2,와이프가 일방적으로 연락을 끊고 짐을 빼버려서 혼자 남겨졌어.,3
3,병원비가 적지 않지만 보험으로 해결되니 다행이야.,4
4,돈을 펑펑 쓰며 사는 자식들이 우리에게 돈을 아끼며 살라고 말했어. 이런 말을 들으...,2
...,...,...
62986,어. 진짜 짜증나게 나한테만 시켜. 나 싫어하는 것 같아.,2
62987,아이를 낳고 아내 건강이 안 좋았는데 점차 좋아지고 있어서 기뻐.,4
62988,직장 일 관련해서 고민되는 일이 있어.,0
62989,그것도 좋겠네. 친구들이랑 약속 좀 잡아야겠다.,4


In [ ]:
dataset_train[1].value_counts()

3    20915
2    12971
4    11090
0     9882
1     8133
Name: 1, dtype: int64

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#tokenize
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data_list, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

warmup_step = int(t_total * warmup_ratio)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-32-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/493 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.5984457731246948 train acc 0.296875
epoch 1 batch id 201 loss 1.3754757642745972 train acc 0.3178249378109453
epoch 1 batch id 401 loss 0.890658438205719 train acc 0.45723581670822944
epoch 1 train acc 0.5021456643002029


<ipython-input-32-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/47 [00:00<?, ?it/s]

...

epoch 29 test acc 0.7762765369045377


  0%|          | 0/493 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.020357497036457062 train acc 0.9921875
epoch 30 batch id 201 loss 0.010703529231250286 train acc 0.9946361940298507
epoch 30 batch id 401 loss 0.026784131303429604 train acc 0.9945254052369077
epoch 30 train acc 0.9946913032454361


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 30 test acc 0.7767752071173036


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("기쁨이")
            

        print(test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
torch.save(model, '/content/drive/Shareddrives/인지프/code/plusdata_ep30_batch_128_traintestdatamodified.h5')

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

분노가 느껴집니다.


슬픔이 느껴집니다.


기쁨이 느껴집니다.


불안이 느껴집니다.


분노가 느껴집니다.


분노가 느껴집니다.


기쁨이 느껴집니다.


분노가 느껴집니다.


